In [2]:
import pandas as pd 
import sqlite3
import seaborn as sns
import datetime
import matplotlib.pyplot as plt 
import numpy as np 
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject


In [4]:
conn = sqlite3.connect('twitch.db')
sql = """
SELECT *
FROM Twitch_Directory
"""
# Read the sql query into pandas to view the results
twitch_data = pd.read_sql_query(sql, conn)
twitch_data.head()

,index,Date,Time,Directory Name,Viewer Count
0,0,2022-01-31,10:45:52,Just Chatting,390431
1,1,2022-01-31,10:45:52,Grand Theft Auto V,239160
2,2,2022-01-31,10:45:52,League of Legends,219827
3,3,2022-01-31,10:45:52,VALORANT,137060
4,4,2022-01-31,10:45:52,Fortnite,115131


In [5]:
twitch_data.dtypes

index              int64
Date              object
Time              object
Directory Name    object
Viewer Count       int64
dtype: object

In [6]:
twitch_data['Directory Name'].unique()

array(['Just Chatting', 'Grand Theft Auto V', 'League of Legends',
       'VALORANT', 'Fortnite', 'Pokémon Legends: Arceus',
       'Counter-Strike: Global Offensive', 'FIFA 22', 'Sports',
       'Apex Legends', 'Dota 2', 'Escape from Tarkov',
       'Dying Light 2: Stay Human', 'Cities: Skylines', 'Rust',
       'Path of Exile', 'Sifu', 'LOST ARK', 'Slots',
       "Tom Clancy's Rainbow Six Siege", 'Minecraft',
       'Call of Duty: Warzone', 'Dead by Daylight', 'Lost Ark',
       'Horizon Forbidden West', 'Mario Kart 8 Deluxe', 'Elden Ring',
       'Destiny 2', 'Politics', 'Rocket League', 'World of Warcraft',
       'Brawl Stars'], dtype=object)

In [7]:
twitch_data['Directory Name'].replace({'LOST ARK' : 'Lost Ark'},  inplace=True)

In [8]:
twitch_data['Directory Name'].unique()

array(['Just Chatting', 'Grand Theft Auto V', 'League of Legends',
       'VALORANT', 'Fortnite', 'Pokémon Legends: Arceus',
       'Counter-Strike: Global Offensive', 'FIFA 22', 'Sports',
       'Apex Legends', 'Dota 2', 'Escape from Tarkov',
       'Dying Light 2: Stay Human', 'Cities: Skylines', 'Rust',
       'Path of Exile', 'Sifu', 'Lost Ark', 'Slots',
       "Tom Clancy's Rainbow Six Siege", 'Minecraft',
       'Call of Duty: Warzone', 'Dead by Daylight',
       'Horizon Forbidden West', 'Mario Kart 8 Deluxe', 'Elden Ring',
       'Destiny 2', 'Politics', 'Rocket League', 'World of Warcraft',
       'Brawl Stars'], dtype=object)

In [9]:
twitch_data.drop(columns=['index'], inplace=True)

In [10]:
twitch_data['Date'] = pd.to_datetime(twitch_data['Date'])

In [11]:
twitch_data['month'] = twitch_data['Date'].dt.strftime('%m')
twitch_data['month'] = twitch_data['month'].astype(int)


In [43]:
twitch_data['Day'] = twitch_data['Date'].dt.strftime('%w')

In [44]:
num_days= {
    '1' : 'Mon', 
    '2' : 'Tue', 
    '3' : 'Wed', 
    '4' : 'Thur', 
    '5' : 'Fri', 
    '6' : 'Sat',
    '0' : 'Sun'
}
twitch_data['Day'].replace(num_days, inplace=True)

In [48]:
twitch_data

,Date,Time,Directory Name,Viewer Count,month,Day
0,2022-01-31,10:45:52,Just Chatting,390431,1,Mon
1,2022-01-31,10:45:52,Grand Theft Auto V,239160,1,Mon
2,2022-01-31,10:45:52,League of Legends,219827,1,Mon
3,2022-01-31,10:45:52,VALORANT,137060,1,Mon
4,2022-01-31,10:45:52,Fortnite,115131,1,Mon
...,...,...,...,...,...,...
355,2022-03-07,10:46:09,Apex Legends,106036,3,Mon
356,2022-03-07,10:46:09,Fortnite,76297,3,Mon
357,2022-03-07,10:46:09,FIFA 22,57183,3,Mon
358,2022-03-07,10:46:09,Counter-Strike: Global Offensive,53727,3,Mon


In [70]:
X = twitch_data.drop(columns=['Date', 'Time', 'Viewer Count'])
y = twitch_data['Viewer Count']

In [71]:
X_train, X_test, y_train, y_test= train_test_split(X, y, random_state=42)

In [72]:
X_train

,Directory Name,month,Day
60,Just Chatting,2,Sun
114,Grand Theft Auto V,2,Fri
258,Fortnite,2,Fri
110,Lost Ark,2,Fri
218,Dota 2,2,Mon
...,...,...,...
71,League of Legends,2,Mon
106,Tom Clancy's Rainbow Six Siege,2,Thur
270,Elden Ring,2,Sun
348,Apex Legends,3,Sun


In [73]:
Ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)

ohe_train = X_train[['Directory Name', 'month']]
ohe_test = X_test[['Directory Name', 'month']]

X_train = Ohe.fit_transform(ohe_train)
X_test = Ohe.transform(ohe_test)

In [74]:
X_train

array([[0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [1., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 1., 0.]])

In [75]:
dt = DecisionTreeRegressor()

dt.fit(X_train, y_train)

print(f'Train: {dt.score(X_train, y_train)}')
print(f'Test: {dt.score(X_test, y_test)}')

Train: 0.7111598069381483
Test: 0.5662584827145769


In [76]:
knn = KNeighborsRegressor()

knn.fit(X_train, y_train)

print(f'Train: {knn.score(X_train, y_train)}')
print(f'Test: {knn.score(X_test, y_test)}')

Train: 0.5893260324532933
Test: 0.37981336456246995


In [77]:
rf = RandomForestRegressor()

rf.fit(X_train, y_train)

print(f'Train: {rf.score(X_train, y_train)}')
print(f'Test: {rf.score(X_test, y_test)}')

Train: 0.7100326170327242
Test: 0.564712698298532


In [78]:
dt_params = {
    'max_features' : ['auto', 'sqrt', 'log2'], 
    'splitter' : ['best', 'random'],
    'max_depth' : [5,10,50], 
}


In [79]:
dt_grid = GridSearchCV(DecisionTreeRegressor(), param_grid=dt_params)

dt_grid.fit(X_train, y_train)

dt_grid.best_params_

{'max_depth': 50, 'max_features': 'auto', 'splitter': 'best'}

In [80]:
dt = DecisionTreeRegressor(max_depth= 50,max_features= 'auto',splitter='random')

dt.fit(X_train, y_train)

print(f'Train: {dt.score(X_train, y_train)}')
print(f'Test: {dt.score(X_test, y_test)}')

Train: 0.7111598069381483
Test: 0.5662033437304932
